# Técnicas Avançadas de **Captura e Tratamento** de Dados

---
## Prof. Bernardo Alves Furtado
---
### MBA em Big Data, Business Analytics e Gestão de Negócios. @**IDP**
---
3 a 21 agosto  -- 21 horas/aula

In [ ]:
# Garantindo que exista a pasta para salvar nossos outputs.
import os
if not os.path.exists('data'):
    os.mkdir('data')

Retomando a partir da aula passada...
#### Link: https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/Aula2.ipynb

## Resolução exercício básico -- Aula 1
1. Leia a tabela disponível em:

https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/exemplo2.csv

5. Padronize os nomes, como feito para o exemplo1
6. Corrija os valores de impostos pagos.
7. Utilize o `describe()` ou outro comando nos impostos corrigidos e identifique a média de impostos pagos.
8. Utilize `sum(coluna)` para o valor total de impostos pagos.
9. O que mais é possível notar de estranho neste exemplo simples?
10. Alguém está familiarizado com o comando `value_counts()` do pandas? Ele pode ajudar a identificar se há valores repetidos (uma de vários jeitos possíveis).

## Respostas

In [15]:
# Lendo o arquivo
import pandas as pd
file = 'https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/exemplo2.csv'
exercicio1 = pd.read_csv(file + '?raw=True')
exercicio1

,cpf,nome,impostos pagos,malha fina?
0,888111000-39,maria João,"R$ 5,43",n
1,111.888.000-93,José maria,"R$ 105.430,70",y
2,18181800093,jose manoel,"R$ 100,00",n
3,18181800194,paula,1285,y
4,18181800194,paula da silva,"R$ 100.000,27",y


In [16]:
# Padronizando os nomes
exercicio1.nome = exercicio1.nome.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
exercicio1.nome = exercicio1.nome.str.lower()
exercicio1

,cpf,nome,impostos pagos,malha fina?
0,888111000-39,maria joao,"R$ 5,43",n
1,111.888.000-93,jose maria,"R$ 105.430,70",y
2,18181800093,jose manoel,"R$ 100,00",n
3,18181800194,paula,1285,y
4,18181800194,paula da silva,"R$ 100.000,27",y


In [19]:
# Corrigindo valores pagos
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return x.replace('R$', '').replace('.', '').replace(',', '.')
    return x

exercicio1['impostos'] = exercicio1['impostos pagos'].apply(clean_currency).astype(float)
print(exercicio1.head())
print(exercicio1['impostos pagos'].dtype)

           cpf            nome impostos pagos malha fina?   impostos
0  88811100039      maria joao        R$ 5,43           n       5.43
1  11188800093      jose maria  R$ 105.430,70           y  105430.70
2  18181800093     jose manoel      R$ 100,00           n     100.00
3  18181800194           paula           1285           y    1285.00
4  18181800194  paula da silva  R$ 100.000,27           y  100000.27
object


In [7]:
# Média impostos pagos
exercicio1.impostos.mean()

41364.28

In [8]:
exercicio1.impostos.sum()

206821.4

In [12]:
# Valores esperados.
v, tt = '41.364,28', '206.821,40'
print(f'Valor médio impostos pagos: {v}.')
print(f'Total: {tt}.')

Valor médio impostos pagos: 41.364,28.
Total: 206.821,40.


### Examinando os itens 9 e 10...

In [ ]:
for col in exercicio1.columns:
    print(f'{exercicio1[col].value_counts()}\n')


### Então? Procedimentos
1. Padronizar CPFs
2. Agregar (`groubpy`) por CPF, somando impostos pagos.
### Sempre muito cuidado com agregação: porque?

In [ ]:
print('')

In [ ]:
"""
Porque agregar sempre implica transformações nas variáveis.
O que fazer com os valores de cada uma das outras colunas no processo de agregação?
Somar, tirar a média? Contar o mais frequente?
"""

In [17]:
# Padronizando CPFs
exercicio1.cpf = exercicio1.cpf.str.replace('-', '').str.replace('.', '').astype(str)
exercicio1

/tmp/ipykernel_22974/2081349549.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  exercicio1.cpf = exercicio1.cpf.str.replace('-', '').str.replace('.', '').astype(str)


,cpf,nome,impostos pagos,malha fina?
0,88811100039,maria joao,"R$ 5,43",n
1,11188800093,jose maria,"R$ 105.430,70",y
2,18181800093,jose manoel,"R$ 100,00",n
3,18181800194,paula,1285,y
4,18181800194,paula da silva,"R$ 100.000,27",y


In [25]:
ex1_agregado = exercicio1.groupby('cpf').agg('sum')
ex1_agregado


,impostos
cpf,
11188800093,105430.70
18181800093,100.00
18181800194,101285.27
88811100039,5.43


In [26]:
ex1_agregado = exercicio1.groupby('cpf').agg('sum').reset_index()
ex1_agregado

,cpf,impostos
0,11188800093,105430.70
1,18181800093,100.00
2,18181800194,101285.27
3,88811100039,5.43


## E agora? O que acontece se eu pedir para 'somar' e 'contar'?

In [ ]:
ex1_agregado = exercicio1.groupby('cpf').agg(['sum', 'count'])
ex1_agregado

---
# $$+$$: Exercícios com pandas. Revisão.

### Dados do Titanic. Análise livre

1. Pergunta: Passageiros de classes inferiores apresentam taxa de sobrevivência menor?

source: https://www.kaggle.com/c/titanic/data

# Kaggle

### "Inside Kaggle you’ll find all the code & data you need to do your data science work. Use over 50,000 public datasets and 400,000 public notebooks to conquer any analysis in no time."

### Dicionário

In [ ]:
'''
Data Dictionary
Variable	Definition	Key
survival 	Survival 	0 = No, 1 = Yes
pclass 	Ticket class 	1 = 1st, 2 = 2nd, 3 = 3rd
sex 	Sex
Age 	Age in years
sibsp 	# of siblings / spouses aboard the Titanic
parch 	# of parents / children aboard the Titanic
ticket 	Ticket number
fare 	Passenger fare
cabin 	Cabin number
embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.
'''

In [28]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
# df = pd.read_csv("https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/titanic_test.csv?raw=True")
df.head()

# source: https://minerandodados.com.br/analise-de-dados-com-python-usando-pandas/

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Aperitivo. Missing dados


In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.Embarked.head()

In [ ]:
df.Cabin.value_counts()

In [ ]:
df.Age.mode()

In [ ]:
df.Fare.mean()

In [ ]:
values = {'Age': df.Age.mode()[0],
          'Cabin': 'SC',
          'Fare': df.Fare.mean()}

df.fillna(value=values, inplace=True)
df.info()

In [30]:
# Replace: male, female
sex = {'male': 0, 'female': 1}
df['sex_dummy'] = df.Sex.replace(sex)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sex_dummy
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [ ]:
df.info()

In [ ]:
cols = [1, 2, 5, 9, 12]
df_simples = df.iloc[:, cols]
df_simples.head()

### Outro jeito de selecionar colunas?

In [31]:
df_simples2 = df[['Survived', 'Pclass', 'Age', 'Fare', 'sex_dummy']]
df_simples2.head()

In [ ]:
corr = df_simples.corr()
corr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# plot the heatmap
sns.heatmap(corr.iloc[:],
            xticklabels=corr.columns,
            yticklabels=corr.columns,
            annot=True,
            cmap='viridis')

# Simple regression and output

In [ ]:
import statsmodels.formula.api as smf

def reg(y, data, colunas=''):
    """ Função que roda as regressões
        Entre com colunas e com base de dados """
    resultado = smf.ols(f"{y} ~  {colunas}", data=data).fit()
    sns.distplot(resultado.resid)
    plt.show()
    return resultado

res = reg('Survived', df_simples, 'Pclass + sex_dummy + Age')
print(res.summary())

---
## Nice output: com cara de paper

Outros resultados com boa apresentação, from Quantative Economics:

https://python.quantecon.org/ols.html


In [ ]:
from statsmodels.iolib.summary2 import summary_col

def print_reg3(m1):
    info_dict={'Log-likelihood': lambda x: f"{x.llf:.2f}",
               'R-squared Adj': lambda x: f"{x.rsquared_adj:.2f}",
               'AIC': lambda x: f"{x.aic:.2f}",
               'BIC': lambda x: f"{x.bic:.2f}",
               'No. observations': lambda x: f"{int(x.nobs):d}"}

    results_table = summary_col(results=[m1],
                                float_format='%0.2f',
                                stars=True,
                                model_names=['Model 1'],
                                info_dict=info_dict)

    results_table.add_title('Table - OLS Regressions')
    print(results_table)

    with open('data/nice_output.txt', 'w') as f:
        f.write(results_table.as_text())
    # return results_table

print_reg3(res)

---
# Exercício extra

0. Simples. Duas modificações apenas.
1. Modifique a função `print_reg3` para que imprima o resultado comparativo de três modelos.
2. Por exemplo, só com **Pclass**, adicionando **Age** e adicionando **sex_dummy**

# Formatos

# Dados tabulares

1. O que são?

In [ ]:
import pandas as pd

Rows, Columns = 'r', 'c'

Qual limitação imediata desse formato?

### Incluem RDBMSs -- Relational DataBase Management Systems

Incluem:

1. CVS -- comma-separated values
2. Qualquer TXT, na verdade ...
3. Planilhas (XLSX, ODS...)

---
# CSV ou TXT

* Delimited: claro, por um delimitador, usualmente
# ','

## Vantagens:
1. Aceito -- *read, write* -- por praticamente todos sistemas/databases/programas
2. Relativamente eficiente -- descrito por um autor como: ***fairly inneficient***

## Desvantagens:
1. Não contém de forma explícita o formato da informação `str, float, int`
2. Como também não o sistema decimal (ponto, vírgula, moedas)...


## Diferenças relevantes em relação ao EXCEL, por exemplo.

1. Alguém?
---

1. O tipo de informação (`float, date`) é (quase) sempre **inferred**
2. Não transparente.
3. Não contém os passos de transformação -- ou seja -- não consegue descrever as alterações feitas
4. Não mantém consistência entre linhas e colunas (enfatiza a questão visual)

---
## Parâmetros de leitura. Opcionais e padrão
# pd.read_csv(filepath)

1. `filepath`: 'endereço'. Obrigatório
2. `sep`: delimitador. Padrão: vírgula
3. `header`: Se utiliza primeira linha para nomes de colunas.
4. Padrão `header=0` (inferidas, a partir da 1a. linha de dados)
5. Utilize `header=None` para não inferir a 1a. linha como títulos de colunas.
6. Caso queira nomear colunas na leitura, use `names=['col1', 'col2']`.
7. Também é possível usar uma coluna específica como nome das colunas: `index_col='nome_col'` ou índice `int` da posição da coluna.
8. Limitar a leitura a algumas colunas: `usecols=['col1', 'col8']
#### Típico para o caso brasileiro
9. `encoding='latin-1'`
10. Para planilhas gigantes: `nrows=100`

source: Confira: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

---
# Fixed-width data

### Que não são delimitados por um separador, mas sim por posição específica na linha.

Vantagem. Compacto

### Exemplo de leitura de dados da amostra do Censo 2010
---

# SQL RDBMS
<span style="color:red">Structured</span> <span style="color:blue">Query</span> Language

Relational DataBase Management Systems

* 1 projeto (arquivo), várias tables. Múltiplas relações.

## Mais restritivas (estruturadas), porém, mais organizadas.

Funções mais relevantes:

1. GROUP BY (agregação)
2. JOIN (relacionamento)
3. WHERE (condição)

### https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [ ]:
sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

# Hierárquicos

![Tree](https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/tree.png?raw=True)

### ""... no specific limit on the depth or names of branches.""
(Mertz, 2021, p.71)

1. Diretórios
2. Redes
3. Grafos (em geral). Matrizes incompletas, esparsas


---
# JSON

### Língua franca HTML, CSS, dados

* JavaScript Object Notation

1. Fácil leitura humanos $$+$$ máquinas
2. Safe. Transparente (distinto de `pickle`, por exemplo)
3. Aceita `false, true, null, objects {}, arrays {}`
4. keys tem que ser **strings**

In [ ]:
import json

exemplo = """
    {"name": "Wes",
    "places_lived": ["United States", "Spain", "Germany"],
    "pet": null,
    "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
    {"name": "Katie", "age": 33, "pet": "Cisco"}]
    }
"""

# source: (McKinney, 2012, p.165)

In [ ]:
j = json.loads(exemplo)
print(type(j))
print(j)

In [ ]:
print(f'Keys: {j.keys()}')
print(f"Content of siblings: {j['siblings']}")
print(f"type {type(j['siblings'])}")

In [ ]:
print(f"{j['siblings'][0].keys()}")
print(f"{j['siblings'][0]['name']}")

### O que aconteceu com `"pet": null`?

In [ ]:
print(j['pet'])

* Observação: é possível usar [schema](https://json-schema.org/) para definir um formato específico de JSON válido para o seu projeto.

---
# Persistência JSON

In [19]:
import json

# Saving
def save_json(ob, filename):
    with open(filename, 'w') as handler:
        json.dump(ob, handler)
    print(f'Saved! You can check {filename}')

# Loading
def load_json(filename):
    with open(filename, "r") as handler:
        ll = json.load(handler)
    print(f'A soma da lista lida é {sum(ll):,.0f}')

In [20]:
l = [x ** 3 for x in range(5)]
f = 'data/my_json.json'
save_json(l, f)

Saved! You can check data/my_json.json


In [21]:
load_json(f)

A soma da lista lida é 100


## Exercício bastante simples

1. Crie uma lista qualquer em `python`.
2. Por exemplo, uma lista com 10 elementos, começando em 100 até 10, de 10 em 10.
*Dica*: a função range admite as opções `range(início, fim, step).
Se o step for negativo, a lista é decrescente, certo?
3. Salve em JSON
4. Load em JSON
5. Calcule a média.

---
# XML
### eXtensible Markup Language

* complex
* semelhante a HTML. Usa tags <> </> para definir os branches.

## Ilustração

In [49]:
# source: adapted from Mertz, 2021, p.99
users = """<?xml version="1.0" encoding="utf-8" ?>
<users>
  <user>
    <name>Mr. John</name>
    <password>26sXFSAFQqlj.*24</password>
    <details>
      <profession employer="IDP">
      Professor</profession>
      <telephone>+55 61 2021 4000</telephone>
    </details>
  </user>
  <user>
    <name>Ms. Bevilacqua</name>
    <password>26sXF584Qqlj.*24</password>
    <details>
      <profession employer="Casa Civil">
      Lobista</profession>
      <telephone>+55 61 1021 4000</telephone>
    </details>
  </user>
</users>
"""

In [50]:
import xml.etree.ElementTree as ET
tree = ET.fromstring(users)

# Let us first find the attributes and text of a profession
user = tree.findall('user')
for each in user:
    print(f"{each.find('name').text}: {each.find('details').find('profession').attrib.get('employer')}")


Mr. John: IDP
Ms. Bevilacqua: Casa Civil


### Exemplo: bookstore
source: https://www.w3schools.com/xml/xml_tree.asp

* root
* parent
* child
* sibling
* attribute
* text

In [42]:
books = """<?xml version="1.0" encoding="UTF-8"?>
<bookstore>
  <book category="cooking">
    <title lang="en">Everyday Italian</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
  <book category="children">
    <title lang="en">Harry Potter</title>
    <author>J K. Rowling</author>
    <year>2005</year>
    <price>29.99</price>
  </book>
  <book category="web">
    <title lang="en">Learning XML</title>
    <author>Erik T. Ray</author>
    <year>2003</year>
    <price>39.95</price>
  </book>
</bookstore>
"""

In [43]:
import xml.etree.ElementTree as ET
books = ET.fromstring(books)
books

<Element 'bookstore' at 0x7f074f109090>

In [ ]:
for child in books:
    print(child.tag, child.attrib)

In [44]:
books = books.findall('book')
books

[<Element 'book' at 0x7f074f109c70>,
 <Element 'book' at 0x7f074f0cb5e0>,
 <Element 'book' at 0x7f074f0cb680>]

In [45]:
[b.find('author').text for b in books]

['Giada De Laurentiis', 'J K. Rowling', 'Erik T. Ray']

In [46]:
[b.find('title').text for b in books]

['Everyday Italian', 'Harry Potter', 'Learning XML']

In [47]:
[b.attrib.get('category') for b in books]

['cooking', 'children', 'web']

In [48]:
[b.find('title').attrib.get('lang') for b in books]

['en', 'en', 'en']

---
# Pickle $$+$$
* Nativo para python
* Prático para projetos próprios, salvando resultados intermediários.
* Não são seguros, se obtidos na *internet* (não-inspecionáveis)
* Salva `python objects`: por exemplo, `funções` e `class` e `class instance`

---
# `class`

In [ ]:
# Cria uma classe de alunos
class Aluno:
    def __init__(self, name):
        self.nome = name
        self.notas = dict()

    def add_course(self, aluno, nome_curso, nota):
        if self.nome == aluno:
            self.notas[nome_curso] = nota

    def __repr__(self):
        return f'{self.notas}'

In [ ]:
# Cria várias instâncias (python objects) de alunos e guarda em um relatório
nomes = ['Paulo', 'Joana']
report = dict()
for nome in nomes:
    report[nome] = Aluno(nome)

print(report)

In [ ]:
# Salva o relatório
import pickle

def save_pickle(ob, name='data/my_first_pickle'):
    with open(name, 'wb') as handler:
        pickle.dump(ob, handler)
    print('Saved!!!')

save_pickle(report)

In [ ]:
# Em outro momento, load o arquivo salvo, atualize as notas
def load_pickle(name='data/my_first_pickle'):
    with open(name, 'rb') as handler:
        loaded_obj = pickle.load(handler)
    print('Objeto lido!')
    return loaded_obj

my_old_report = load_pickle()
print(my_old_report)

In [ ]:
# Adicione notas
notas = [('Paulo', 10), ('Joana', 10)]
for nota in notas:
    nome = nota[0]
    my_old_report[nome].add_course(nome, 'TACD', nota[1])

print(my_old_report)


---
# Exercício

1. Crie uma classe simples, como o exemplo de Aluno
2. Chame-a de **Pet**, por exemplo e dê uma _id ou nome, ou ambos
3. Utilize somente a primeira função `__init__` que é construtora
4. Se a _id for numérica, crie 10 objetos do tipo Pet

In [ ]:
# Exemplo
try:
    my_pets = [Pet(i) for i in range(10)]
except NameError:
    print("Essa função só funciona se houver a `class` Pet() com uma função __init__ que recebe uma id")

5. Criado o objeto my_pets, salve ele em PICKLE
6. Não será possível salvar em JSON. Nesse caso, crie um dicionário simples e salve em JSON
7. Load o PICKLE que você salvou e inspecione o objeto.
8. Abra o arquivo JSON no diretório que você salvou e inspecione o objeto.

---
# Aperitivo II
source: https://open.fda.gov/apis/

### API is an acronym for Application Programming Interface. <br>

### An **API call** is any request sent to the API.

### Requests are typically sent to the API in one of two ways:

1. Manually using a web browser
(such as navigating to the URL https://api.fda.gov/drug/label.json) or

2. **Programmatically** sending the request via executing code that **sends the API call** and processes the response. Continue reading this documentation for more details on how to compose an API call for openFDA specifically.

### The API **returns individual results as JSON** by default.

### The JSON object has two sections:

   1. `meta`: Metadata about the query, including a disclaimer, link to data license, last-updated date, and total matching records, if applicable.

   2. `results`: An **array** of matching results, dependent on which endpoint was **queried**.



# SQL
source: https://www.sqlitetutorial.net/sqlite-python
### Exemplo básico

1. Cria database
2. Faz conexão


In [ ]:
import sqlite3

# Endereço no qual irá guardar a database
my_first_db = 'data/my_db.db'

def connect_db(database):
    con = sqlite3.connect(database)
    print(f'Conexão realizada com sucesso. Versão sqlite: {sqlite3.version}')
    return con

3. Cria planilhas -- tables. Isso só é feito no início do projeto
    1. Criaremos uma planilha `projects` e uma `tasks`

In [ ]:
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

4. Em python, usamos o `with` toda vez que um processo precisa de `open` e depois `close`
O `with` cria um contexto no qual o `handler`, a `conexão` são válidos!

In [ ]:
with connect_db(my_first_db) as conn:
    conn.execute(sql_create_projects_table)
    conn.execute(sql_create_tasks_table)
    conn.commit()

5. Agora, existe uma database, com duas planilhas formatadas.
6. Estamos prontos para criar um projeto, com tarefas.
7. As duas funções abaixo criam o processo de inserção de projetos e tarefas nas planilhas.

In [ ]:
def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid


def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

8. Agora, já temos a database, as planilhas e as funções que inserem informações.
9. Vamos começar a inserir, de fato, detalhes dos projetos.

In [ ]:
with connect_db(my_first_db) as conn:
    # create a new project
    project = ('Primeiro Cool App com SQLite & Python', '2021-07-21', '2021-12-31');
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Rever com cuidado e treinar as aulas Prof. Bernardo', 1, 1, project_id, '2021-08-03', '2021-08-21')
    task_2 = ('Realizar exercícios que extrapolam os exemplos básicos', 2, 1, project_id, '2021-08-03', '2021-08-21')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)

10. Se você rodar esse comando em uma IDE de python. Tipo PyCharm community, my favorite.
11. Então pode ir ao terminal
    1. Enter `sqlite3 data/my_db.db`
    2. `.header on`
    3. `.mode column`
    4. `select * from tasks;`
    5. `.exit`

![sql exemplo](https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/sql_exemplo.png?raw=True)

12. Alternativamente, novamente usando o `pandas`, você pode fazer:

In [ ]:
with connect_db(my_first_db) as conn:
    data = pd.read_sql_query("SELECT * FROM tasks", conn)

print(data)

In [ ]:
print(data.info())

# Exercício SQL.
### Vamos repetir exatamente o mesmo processo do exemplo acima, porém, no próprio ambiente e mudando algumas configurações:

1. (Instale, se necessário) e import no seu ambiente a biblioteca `sqlite3`.
2. Crie um local/nome para a sua primeira database.
3. Utilize a mesma função **connect_db()**
    1. Pode alterar o nome da função, se quiser,
    2. bem como, a mensagem do `print`
4. Verifique se vai fazer alguma alteração nas colunas e nos nomes das suas tabelas.
5. Crie a o texto original para criar as duas tabelas.
6. Efetivamente, use `conexao.execute()` para criar a database e as duas tabelas.
7. Verifique, de acordo com sua possível mudança de colunas nas tabelas, se as funções `create_project` e
 `create_task` precisam ser alteradas.
8. Insira alguns elementos na sua nova database.
9. Leia a nova database com `pandas` para verificarmos todo o processo.